In [2]:
#Imports
import json
import nltk
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
import pandas as pd
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [3]:
#Load models Sentiment
nltk.download('vader_lexicon')
nltk.download('punkt')
sentimentIntensityAnalyzer = SentimentIntensityAnalyzer()


pretrained_LM_path = "kornosk/bert-election2020-twitter-stance-biden-KE-MLM"

#Load models Stance
tokenizer = AutoTokenizer.from_pretrained(pretrained_LM_path)
stanceDetectionmodel = AutoModelForSequenceClassification.from_pretrained(pretrained_LM_path)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/annafernandezrajal/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/annafernandezrajal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
#Settings
pd.set_option('display.width', 1000)

In [5]:
#Load data
twitter_data = pd.read_csv('dataset/train.csv', encoding="unicode_escape", lineterminator='\r')

twitter_data.head(10)

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos
5,Hillary can not win. Here's hoping the Dems of...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
6,Respect FOR the law and respect BY the law Yes...,Hillary Clinton,NONE,2. The tweet does NOT expresses opinion about ...,pos
7,I don't want to be appointed to an Ambassador ...,Hillary Clinton,NONE,2. The tweet does NOT expresses opinion about ...,neg
8,#StopHillary2016 @HillaryClinton if there was ...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
9,@HillaryClinton End lawless #ClintonFoundation...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg


In [6]:
#Calculate Sentiment
def calculateSentiment(text):
    return sentimentIntensityAnalyzer.polarity_scores(text)['compound']

twitter_data['sentimentVader'] = twitter_data['Tweet'].apply(calculateSentiment)
display(twitter_data)

,Tweet,Target,Stance,Opinion Towards,Sentiment,sentimentVader
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,0.4019
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos,0.8126
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,0.3612
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,-0.5267
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos,0.3612
...,...,...,...,...,...,...
2909,"There's a law protecting unborn eagles, but no...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,0.1139
2910,I am 1 in 3... I have had an abortion #Abortio...,Legalization of Abortion,AGAINST,2. The tweet does NOT expresses opinion about ...,other,0.0000
2911,How dare you say my sexual preference is a cho...,Legalization of Abortion,AGAINST,2. The tweet does NOT expresses opinion about ...,neg,0.0000
2912,"Equal rights for those 'born that way', no rig...",Legalization of Abortion,AGAINST,2. The tweet does NOT expresses opinion about ...,neg,-0.2960


In [7]:
# Calculate Stance Detection

id2label = {
    0: "AGAINST",
    1: "FAVOR",
    2: "NONE"
}

def calculateStance(text):
    inputs = tokenizer(text, return_tensors="pt")
   # outputs = stanceDetectionmodel(**inputs)
    print(inputs)
    #prediction = torch.softmax(outputs[0], dim=1)[0].tolist()
    return 0#id2label[np.argmax(prediction)]


twitter_data['stancePrediction'] = twitter_data['Tweet'].apply(calculateStance)


Intel MKL FATAL ERROR: This system does not meet the minimum requirements for use of the Intel(R) Math Kernel Library.
The processor must support the Intel(R) Supplemental Streaming SIMD Extensions 3 (Intel(R) SSSE3) instructions.
The processor must support the Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) instructions.
The processor must support the Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


: 

: 

In [ ]:
display(twitter_data)

,Tweet,Target,Stance,Opinion Towards,Sentiment,sentimentVader,stancePrediction
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,0.4019,"[input_ids, token_type_ids, attention_mask]"
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos,0.8126,"[input_ids, token_type_ids, attention_mask]"
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,0.3612,"[input_ids, token_type_ids, attention_mask]"
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,-0.5267,"[input_ids, token_type_ids, attention_mask]"
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos,0.3612,"[input_ids, token_type_ids, attention_mask]"
...,...,...,...,...,...,...,...
2909,"There's a law protecting unborn eagles, but no...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,0.1139,"[input_ids, token_type_ids, attention_mask]"
2910,I am 1 in 3... I have had an abortion #Abortio...,Legalization of Abortion,AGAINST,2. The tweet does NOT expresses opinion about ...,other,0.0000,"[input_ids, token_type_ids, attention_mask]"
2911,How dare you say my sexual preference is a cho...,Legalization of Abortion,AGAINST,2. The tweet does NOT expresses opinion about ...,neg,0.0000,"[input_ids, token_type_ids, attention_mask]"
2912,"Equal rights for those 'born that way', no rig...",Legalization of Abortion,AGAINST,2. The tweet does NOT expresses opinion about ...,neg,-0.2960,"[input_ids, token_type_ids, attention_mask]"
